In [3]:
%load_ext dotenv
%reload_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [27]:
from qiskit_optimization.runtime import QAOAClient
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp

In [4]:
import os
from qiskit import IBMQ
IBMQ.enable_account(os.environ["IBMQ_TOKEN"])

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
provider = IBMQ.get_provider(hub="ibm-q", group="open", project="main")


In [32]:
qaoa_mes = QAOAClient(
    provider=provider, backend=provider.get_backend("simulator_stabilizer"), reps=2, alpha=0.75
)

qaoa = MinimumEigenOptimizer(qaoa_mes)

In [23]:
from src.quadratic_solver import quadratic_model, load_linear_prog_object

In [24]:
qubo = quadratic_model("lp_tiny.pkl")
linear = load_linear_prog_object("lp_tiny.pkl")

In [28]:
len(list(qubo.iter_variables()))
qubo_qiskit = from_docplex_mp(qubo)
# len(list(linear.iter_variables()))

In [33]:
result = qaoa.solve(qubo_qiskit)
print(result.prettyprint())

RuntimeError: The job cdhs3856oa47pjs0th0g failed unexpectedly.